# Projeto do Nanodegree - 2025/1

## Introdução

Este notebook faz parte do projeto do Nanodegree 2025/1 da disciplina de Machine Learning & Inteligência Artificial.
Seu objetivo é analisar, explorar, preparar os dados e aplicar a modelos para a tarefa de previsão de evasão de estudantes em um curso online síncrono promovido pela PensComp.

In [ ]:
pip install unidecode shap

## 1. Configuração do Ambiente

### 1.1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode
import shap

# Modelagem e Métricas
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### 1.2. Definição de Constantes

Centralizar constantes melhora a legibilidade e facilita a manutenção do código. Se precisarmos ajustar um parâmetro (como o número de dias para considerar evasão), podemos fazer isso em um único lugar.

In [ ]:
# Constantes do Projeto
DIAS_INATIVIDADE_EVASAO = 60
LIMITE_NAN_DROP = 0.70 # Limite de 70% de valores nulos para remover uma coluna
RANDOM_STATE = 42 # Semente para garantir a reprodutibilidade dos resultados

### 1.3. Funções Auxiliares

Criar funções para tarefas repetitivas (como avaliar modelos) torna o notebook mais limpo e segue o princípio DRY (Don't Repeat Yourself).

In [ ]:
def avaliar_modelo(nome_modelo, y_verdadeiro, y_previsao, ax=None):
    """
    Imprime o relatório de classificação e plota a matriz de confusão para um modelo.
    """
    print(f"\n--- Relatório de Classificação: {nome_modelo} ---")
    print(classification_report(y_verdadeiro, y_previsao))

    # Plotar matriz de confusão
    cm = confusion_matrix(y_verdadeiro, y_previsao)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    
    if ax:
        disp.plot(cmap=plt.cm.Blues, ax=ax, colorbar=False)
        ax.set_title(f'Matriz de Confusão - {nome_modelo}')
    else:
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Matriz de Confusão - {nome_modelo}')
        plt.show()

## 2. Carga e Análise Exploratória dos Dados (EDA)

In [ ]:
df = pd.read_csv('dados_projeto_evasao_treino - Copia.csv')

In [ ]:
df.info()

### 2.1. Limpeza e Preparação dos Dados

In [ ]:
# Remover colunas de ID e alunos que nunca acessaram
df = df.drop("Unnamed: 0", axis=1)
df = df[df['ts_primeiro_acesso'] != 0]

# Converter timestamps para datetime
df['ts_primeiro_acesso'] = pd.to_datetime(df['ts_primeiro_acesso'], unit='s')
df['ts_ultimo_acesso'] = pd.to_datetime(df['ts_ultimo_acesso'], unit='s')

# Limpeza da coluna de cidade
df['ds_cidade_usuario'] = (
    df['ds_cidade_usuario']
    .astype(str)
    .str.strip()
    .str.upper()
    .apply(unidecode.unidecode)
)
df['ds_cidade_usuario'].replace("NAN", "CIDADE NAO INFORMADA", inplace=True)

### 2.2. Tratamento de Valores Ausentes
A estratégia de tratamento de valores nulos é crucial. Removemos colunas com excesso de dados faltantes (>70%) e preenchemos as demais com base em hipóteses de negócio (ex: nulo em engajamento significa atividade zero) ou estatísticas robustas (mediana).

In [ ]:
# Identificar colunas a serem removidas com base no limiar
missing_ratio = df.isnull().mean()
colunas_para_remover = missing_ratio[missing_ratio > LIMITE_NAN_DROP].index
df.drop(columns=colunas_para_remover, inplace=True)
print(f"Colunas removidas por excesso de NAs: {list(colunas_para_remover)}")

# Preenchimento com zero (ausência de atividade)
cols_fill_zero = ["vl_desempenho_questionario", "vl_engajamento_usuario_por_intervalo", 
                  "vl_engajamento_usuario_intradia", "vl_desempenho_usuario", "vl_media_notas"]
for col in cols_fill_zero:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Preenchimento com mediana (variáveis de tempo contínuas)
cols_fill_median = ["vl_medio_tempo_questionario", "vl_medio_tempo_questionario_avaliado"]
for col in cols_fill_median:
    if col in df.columns:
      median_val = df[col].median()
      df[col] = df[col].fillna(median_val)

### 2.3. Análise Visual dos Dados

#### Distribuição do Desempenho Geral e Dias de Inatividade

Analisar a distribuição destas duas variáveis é fundamental, pois elas formam a base da nossa definição de evasão. Notamos uma alta concentração de alunos com desempenho zero e um grande número de alunos inativos há muito tempo.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de Desempenho
sns.histplot(df['vl_desempenho_usuario'], bins=20, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Distribuição do Desempenho dos Alunos')
axes[0].set_xlabel('Desempenho')
axes[0].set_ylabel('Frequência')

# Gráfico de Dias desde o Último Acesso
sns.histplot(df['nr_dias_desde_ultimo_acesso'], bins=30, kde=True, ax=axes[1], color='salmon')
axes[1].set_title('Distribuição dos Dias Desde o Último Acesso')
axes[1].set_xlabel('Dias Desde o Último Acesso')
axes[1].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

#### Heatmap de Correlação

O heatmap nos ajuda a identificar relações lineares entre as variáveis. Correlações fortes podem indicar redundância de features, enquanto a correlação com a variável-alvo (que criaremos a seguir) é um forte indicador de poder preditivo.

In [ ]:
df_numeric = df.select_dtypes(include=np.number)

plt.figure(figsize=(14, 10))
sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm', fmt='.2f', annot_kws={'size': 8})
plt.title('Heatmap de Correlação entre Variáveis Numéricas')
plt.show()

## 3. Engenharia de Features

Nesta etapa, criamos novas variáveis a partir dos dados existentes para capturar informações de negócio relevantes e acionáveis.

### 3.1. Criação da Variável Alvo (`evadiu`)

Definimos uma regra de negócio para classificar um aluno como evadido. Essa será a variável que nossos modelos tentarão prever.

> **Regra:** Se o aluno tem **desempenho zero** E está **inativo há mais de 60 dias**, ele é classificado como `evadiu = 1`.

In [ ]:
df['evadiu'] = ((df['vl_desempenho_usuario'] == 0) & (df['nr_dias_desde_ultimo_acesso'] > DIAS_INATIVIDADE_EVASAO)).astype(int)
print("Distribuição da variável 'evadiu':")
print(df['evadiu'].value_counts(normalize=True))

### 3.2. Criação de Perfis de Risco

Para uma análise mais estratégica, segmentamos os alunos em perfis de risco. Isso permite que a PensComp direcione ações específicas para cada grupo, em vez de uma abordagem genérica.

In [ ]:
def criar_perfil(row):
    if row["vl_desempenho_usuario"] == 0 and row["nr_dias_desde_ultimo_acesso"] > DIAS_INATIVIDADE_EVASAO:
        return "Alto Risco (Evasão)"
    elif row["vl_desempenho_usuario"] > 0.7 and row["nr_dias_desde_ultimo_acesso"] > DIAS_INATIVIDADE_EVASAO:
        return "Reengajamento Urgente"
    elif row["vl_desempenho_usuario"] < 0.3 and row['nr_dias_desde_ultimo_acesso'] <= DIAS_INATIVIDADE_EVASAO:
        return "Apoio Pedagógico"
    else:
        return "Estável"

df["perfil"] = df.apply(criar_perfil, axis=1)

#### Visualização da Distribuição dos Perfis

Visualizar a contagem de alunos em cada perfil nos dá uma noção clara da urgência e do tamanho de cada segmento.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(y='perfil', data=df, order=df['perfil'].value_counts().index, palette='viridis')
plt.title('Contagem de Alunos por Perfil de Risco')
plt.xlabel('Quantidade de Alunos')
plt.ylabel('Perfil')
plt.show()

## 4. Pré-Processamento e Modelagem

### 4.1. Preparação dos Dados para os Modelos

Aqui, preparamos os conjuntos de dados `X` (features) e `y` (alvo) e os padronizamos para que os modelos de Machine Learning possam processá-los corretamente.

In [ ]:
# Seleção do Alvo (y) e Features (X)
y = df["evadiu"]
X = df.drop(columns=["evadiu", "perfil", "ds_cidade_usuario"], errors="ignore")

# Remover colunas de data/hora, pois os modelos não as processam diretamente
X = X.select_dtypes(exclude=["datetime64[ns]"])

# Padronização das Features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Divisão em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

### 4.2. Treinamento e Avaliação dos Modelos

#### Modelo 1: Regressão Logística

Começamos com a Regressão Logística por ser um modelo simples, rápido e altamente interpretável, servindo como um excelente baseline.

In [ ]:
log_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

#### Modelo 2: Random Forest

Em seguida, usamos o Random Forest, um modelo de ensemble robusto que captura relações não-lineares e geralmente oferece maior performance.

In [ ]:
rf_model = RandomForestClassifier(random_state=RANDOM_STATE)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

#### Modelo 3: Rede Neural

Por fim, implementamos uma Rede Neural simples, capaz de aprender padrões complexos, para buscar a máxima performance.

In [ ]:
# Arquitetura do modelo
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = nn_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop],
    verbose=0 # Silenciar output do treino
)

y_pred_nn_prob = nn_model.predict(X_test)
y_pred_nn = (y_pred_nn_prob > 0.5).astype("int32")

### 4.3. Comparação dos Resultados e Matrizes de Confusão

Com a função `avaliar_modelo`, podemos comparar de forma limpa e direta a performance dos três modelos.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

avaliar_modelo("Regressão Logística", y_test, y_pred_log, ax=axes[0])
avaliar_modelo("Random Forest", y_test, y_pred_rf, ax=axes[1])
avaliar_modelo("Rede Neural", y_test, y_pred_nn, ax=axes[2])

axes[1].set_ylabel("")
axes[2].set_ylabel("")

plt.tight_layout()
plt.show()

### 4.4. Nota Importante sobre a Performance Perfeita

Observamos que todos os modelos, especialmente a Rede Neural, alcançaram 100% de precisão, recall e acurácia no conjunto de teste. Embora pareça um resultado ideal, é fundamental entender a causa.

**Isso não é um erro, mas uma consequência do desenho do problema.**

A variável-alvo `evadiu` foi criada com uma regra determinística baseada em `vl_desempenho_usuario` e `nr_dias_desde_ultimo_acesso`. Como essas mesmas variáveis (ou outras fortemente correlacionadas) estão presentes nas features de treino, os modelos aprenderam a replicar perfeitamente essa regra.

**Conclusão Prática:** O modelo é extremamente eficaz para **classificar o perfil atual de um aluno** e identificar quem já se encaixa nos critérios de evasão. No entanto, ele não está *prevendo* uma evasão futura, mas sim *identificando* um estado atual.

### 4.5. Validação Cruzada

Para confirmar a estabilidade dos modelos, aplicamos a Validação Cruzada. Isso nos mostra se a alta performance se mantém em diferentes subconjuntos dos dados.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

scores_log = cross_val_score(log_model, X_scaled, y, cv=cv, scoring="f1_weighted")
scores_rf = cross_val_score(rf_model, X_scaled, y, cv=cv, scoring="f1_weighted")

print("\nF1-score Ponderado (Validação Cruzada)")
print(f"Regressão Logística: {scores_log.mean():.4f} ± {scores_log.std():.4f}")
print(f"Random Forest: {scores_rf.mean():.4f} ± {scores_rf.std():.4f}")

## 5. Interpretabilidade do Modelo com SHAP (XAI)

Usamos o SHAP para entender *quais* features são mais importantes para as decisões do modelo mais robusto (Random Forest). Isso adiciona transparência e confiança aos resultados.

In [ ]:
explainer_rf = shap.TreeExplainer(rf_model)
shap_values_rf = explainer_rf.shap_values(X_test)

shap.summary_plot(shap_values_rf[1], X_test, feature_names=X_test.columns, plot_type='bar', title='Importância Global das Features (Random Forest)')

O gráfico SHAP confirma nossa análise: `vl_desempenho_usuario` e `nr_dias_desde_ultimo_acesso` são as variáveis mais impactantes, pois são as que definem a própria evasão. Isso valida que o modelo aprendeu corretamente a lógica de negócio.

## 6. Conclusões e Próximos Passos

### Conclusões do Projeto

1.  **Modelos Eficazes:** Conseguimos construir modelos de Machine Learning que classificam com 100% de acerto os alunos que se encaixam na definição de evasão estabelecida pelo negócio (desempenho zero e inatividade superior a 60 dias).
2.  **Validação da Lógica de Negócio:** A análise de interpretabilidade (SHAP) confirmou que as variáveis de desempenho e inatividade são, de fato, os fatores decisivos, validando a regra de negócio criada.
3.  **Segmentação Acionável:** A criação de "Perfis de Risco" permite que a PensComp vá além da simples classificação e adote estratégias focadas para cada segmento de alunos, como campanhas de reengajamento ou oferta de apoio pedagógico.


### Próximos Passos: Evoluindo para um Modelo Preditivo

O maior valor para o negócio está em **prever a evasão antes que ela aconteça**. Para evoluir este projeto de um classificador de estado atual para um modelo preditivo proativo, os próximos passos seriam:

1.  **Reformular o Problema (Engenharia de Features Temporal):**
    * **Definir uma Janela de Observação:** Usar apenas os dados das primeiras **2 ou 3 semanas** de atividade de cada aluno.
    * **Definir uma Janela de Predição:** Criar a variável-alvo `evadira_nos_proximos_60_dias`, que seria `1` se o aluno, *após* a janela de observação, ficasse inativo.

2.  **Ajuste Fino de Hiperparâmetros:**
    * Utilizar técnicas como `GridSearchCV` ou `RandomizedSearchCV` para encontrar a melhor combinação de hiperparâmetros para os modelos (ex: `max_depth`, `n_estimators` no Random Forest), otimizando a performance preditiva.

3.  **Implantação (Deploy):**
    * Empacotar o modelo treinado em uma API (usando Flask ou FastAPI) para que possa ser consumido por outros sistemas, permitindo, por exemplo, a criação de um painel de controle em tempo real que alerte sobre alunos em risco.